# 坚持

许多人工智能应用程序需要内存来在单个会话“线程”中的多个交互之间共享上下文。
在 LangGraph 中，这种类型的会话级内存可以使用以下方式添加到任何图形中：
[检查点](https://langchain-ai.github.io/langgraphjs/reference/interfaces/index.Checkpoint.html)。

只需使用兼容的检查点编译图表即可。下面是使用简单内存中“MemorySaver”的示例：
```javascript
import { MemorySaver } from "@langchain/langgraph";

const checkpointer = new MemorySaver();
const graph = workflow.compile({ checkpointer });
```
本指南展示了如何将线程级持久性添加到图表中。

<div class="警告提示">
<p class="admonition-title">注意：多对话记忆</p>
<p>
如果您需要在多个对话或用户之间<b>共享</b>内存（跨线程持久性），请查看此<a href="https://langchain-ai.github.io/langgraphjs/how-tos/cross-thread-persistence/">操作指南</a>）。
</p>
</div>

<div class="警告提示">
<p class="admonition-title">注意</p>
<p>
在本指南中，我们将从头开始创建透明（但冗长）的代理。您可以使用 <code>createReactAgent(model, tools=tool, checkpointer=checkpointer)</code> (<a href="https://langchain-ai.github.io/langgraphjs/reference/functions/prebuilt.createReactAgent.html">API doc</a>) 构造函数完成类似的功能。如果你习惯了 LangChain 的 <a href="https://js.langchain.com/docs/how_to/agent_executor">AgentExecutor</a> 类，这可能更合适。
</p>
</div>

## 设置

本指南将使用 OpenAI 的 GPT-4o 模型。我们可以选择设置 API 密钥
对于 [LangSmith 追踪](https://smith.langchain.com/)，这将为我们提供
一流的可观测性。

In [1]:
// process.env.OPENAI_API_KEY = "sk_...";

// 可选，在 LangSmith 中添加跟踪
// process.env.LANGCHAIN_API_KEY = "ls__...";
process.env.LANGCHAIN_CALLBACKS_BACKGROUND = "true";
process.env.LANGCHAIN_TRACING_V2 = "true";
process.env.LANGCHAIN_PROJECT = "Persistence: LangGraphJS";

Persistence: LangGraphJS


## 定义状态

状态是图中所有节点的接口。


In [2]:
import { Annotation } from "@langchain/langgraph";
import { BaseMessage } from "@langchain/core/messages";

const GraphState = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
});

## 设置工具

我们将首先定义我们想要使用的工具。对于这个简单的例子，我们将
使用创建占位符搜索引擎。然而，创建起来确实很容易
您自己的工具 - 请参阅文档
[此处](https://js.langchain.com/docs/how_to/custom_tools) 了解如何操作
那。

In [3]:
import { tool } from "@langchain/core/tools";
import { z } from "zod";

const searchTool = tool(async ({}: { query: string }) => {
  // 这是实际实现的占位符
  return "Cold, with a low of 13 ℃";
}, {
  name: "search",
  description:
    "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
  schema: z.object({
    query: z.string().describe("The query to use in your search."),
  }),
});

await searchTool.invoke({ query: "What's the weather like?" });

const tools = [searchTool];

我们现在可以将这些工具包装在一个简单的
[ToolNode](/langgraphjs/reference/classes/langgraph_prebuilt.ToolNode.html)。
每当工具（函数）被调用时，该对象就会实际运行它们
我们的法学硕士。

In [4]:
import { ToolNode } from "@langchain/langgraph/prebuilt";

const toolNode = new ToolNode(tools);

## 设置模型

现在我们将加载
[聊天模型](https://js.langchain.com/docs/concepts/#chat-models)。

1.它应该与消息一起使用。我们将以表格形式代表所有代理状态
消息，因此它需要能够与它们很好地配合。
2.它应该与
[工具调用](https://js.langchain.com/docs/how_to/tool_calling/#passing-tools-to-llms),
这意味着它可以在响应中返回函数参数。

<div class="警告提示">
<p class="admonition-title">注意</p>
<p>
这些模型要求不是使用 LangGraph 的一般要求 - 它们只是这个示例的要求。
</p>
</div>

In [5]:
import { ChatOpenAI } from "@langchain/openai";

const model = new ChatOpenAI({ model: "gpt-4o" });

完成此操作后，我们应该确保模型知道它具有这些
可供调用的工具。我们可以通过调用来做到这一点
[bindTools](https://v01.api.js.langchain.com/classes/langchain_core_language_models_chat_models.BaseChatModel.html#bindTools)。

In [6]:
const boundModel = model.bindTools(tools);

## 定义图表

我们现在可以把它们放在一起。我们将首先在没有检查点的情况下运行它：


In [7]:
import { END, START, StateGraph } from "@langchain/langgraph";
import { AIMessage } from "@langchain/core/messages";
import { RunnableConfig } from "@langchain/core/runnables";

const routeMessage = (state: typeof GraphState.State) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1] as AIMessage;
  // 如果没有调用任何工具，我们就可以完成（响应用户）
  if (!lastMessage.tool_calls?.length) {
    return END;
  }
  // 否则，如果有，我们继续并调用工具
  return "tools";
};

const callModel = async (
  state: typeof GraphState.State,
  config?: RunnableConfig,
) => {
  const { messages } = state;
  const response = await boundModel.invoke(messages, config);
  return { messages: [response] };
};

const workflow = new StateGraph(GraphState)
  .addNode("agent", callModel)
  .addNode("tools", toolNode)
  .addEdge(START, "agent")
  .addConditionalEdges("agent", routeMessage)
  .addEdge("tools", "agent");

const graph = workflow.compile();

In [8]:
let inputs = { messages: [{ role: "user", content: "Hi I'm Yu, nice to meet you." }] };
for await (
  const { messages } of await graph.stream(inputs, {
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

Hi I'm Yu, nice to meet you.
-----

Hi Yu! Nice to meet you too. How can I assist you today?
-----



In [9]:
inputs = { messages: [{ role: "user", content: "Remember my name?" }] };
for await (
  const { messages } of await graph.stream(inputs, {
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

Remember my name?
-----

You haven't shared your name with me yet. What's your name?
-----



## 添加内存

让我们用检查点再试一次。我们将使用
[MemorySaver](/langgraphjs/reference/classes/index.MemorySaver.html),
这将在内存中“保存”检查点。

In [10]:
import { MemorySaver } from "@langchain/langgraph";

// 这里我们只保存在内存中
const memory = new MemorySaver();
const persistentGraph = workflow.compile({ checkpointer: memory });

In [11]:
let config = { configurable: { thread_id: "conversation-num-1" } };
inputs = { messages: [{ role: "user", content: "Hi I'm Jo, nice to meet you." }] };
for await (
  const { messages } of await persistentGraph.stream(inputs, {
    ...config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

Hi I'm Jo, nice to meet you.
-----

Hello Jo, nice to meet you too! How can I assist you today?
-----



In [12]:
inputs = { messages: [{ role: "user", content: "Remember my name?"}] };
for await (
  const { messages } of await persistentGraph.stream(inputs, {
    ...config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

Remember my name?
-----

Yes, I'll remember that your name is Jo. How can I assist you today?
-----



## 新对话主题

如果我们想开始一个新的对话，我们可以传入一个不同的
**[[代码0]]**。噗！所有的记忆都消失了（开玩笑，他们永远都会
住在另一个线程中）！


In [13]:
config = { configurable: { thread_id: "conversation-2" } };

{ configurable: { thread_id: 'conversation-2' } }


In [14]:
inputs = { messages: [{ role: "user", content: "you forgot?" }] };
for await (
  const { messages } of await persistentGraph.stream(inputs, {
    ...config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

you forgot?
-----



Could you please provide more context or details about what you are referring to? This will help me assist you better.
-----

